# Part 2b: Add Validation

Load and process patient data with BMI calculations.

**Your task:** Add schema and bounds validation to catch data quality issues early.

---

## Load data

In [1]:
# Import libraries
import pandas as pd

# validate_schema(df, required_columns)
def validate_schema(df, required_columns):
    missing = [columns for columns in required_columns if columns not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    
# validate_bounds(df, bounds_dict)
def validate_bounds(df, bounds_dict):
    for col, min_max in bounds_dict.items():
        min_val, max_val = min_max
        oob = ~df[col].between(min_val, max_val)
        if oob.any():
            bad_rows = df.loc[oob, ["patient_id", col]]
            raise ValueError(f" Out-of-bounds values in column:'{col};\n Problematic rows: '{bad_rows}'")


df = pd.read_csv("data/patient_intake.csv")

# Call validate_schema() to check for required columns: ["patient_id", "weight_kg", "height_cm", "age"]
validate_schema(df, ["patient_id", "weight_kg", "height_cm", "age"])

# Call validate_bounds() with bounds: weight_kg: (30, 250) | height_cm: (120, 230) | age: (0, 110)
validate_bounds(df, {
    "weight_kg": (30, 250),
    "height_cm": (120, 230),
    "age": (0, 110)
})


df.head()

,patient_id,first_name,last_name,weight_kg,height_cm,age,sex
0,P001,Mark,Johnson,91.5,177,46,M
1,P002,Donald,Walker,80.5,164,29,M
2,P003,Nancy,Rhodes,74.3,163,47,F
3,P004,Steven,Miller,64.4,171,71,M
4,P005,Javier,Johnson,72.8,178,18,M


---

## Calculate BMI

In [2]:
df["height_m"] = df["height_cm"] / 100
df["bmi"] = df["weight_kg"] / (df["height_m"] ** 2)
df["bmi"] = df["bmi"].round(1)

df[["patient_id", "weight_kg", "height_cm", "bmi"]].head()

,patient_id,weight_kg,height_cm,bmi
0,P001,91.5,177,29.2
1,P002,80.5,164,29.9
2,P003,74.3,163,28.0
3,P004,64.4,171,22.0
4,P005,72.8,178,23.0


---

## Categorize BMI

In [3]:
df["bmi_category"] = pd.cut(
    df["bmi"],
    bins=[0, 18.5, 25, 30, float("inf")],
    labels=["Underweight", "Normal", "Overweight", "Obese"],
    right=False
)

df[["patient_id", "bmi", "bmi_category"]].head()

,patient_id,bmi,bmi_category
0,P001,29.2,Overweight
1,P002,29.9,Overweight
2,P003,28.0,Overweight
3,P004,22.0,Normal
4,P005,23.0,Normal


---

## Summary statistics

In [4]:
summary = df.groupby("bmi_category")["patient_id"].count()
print("\nBMI category distribution:")
print(summary)

high_risk = df[df["bmi"] > 30]
print(f"\nHigh-risk patients (BMI > 30): {len(high_risk)}")


BMI category distribution:
bmi_category
Underweight     0
Normal         15
Overweight     21
Obese          14
Name: patient_id, dtype: int64

High-risk patients (BMI > 30): 14


/var/folders/q2/425x2zhd1b3f3hnjp4v223880000gn/T/ipykernel_87844/1212969269.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby("bmi_category")["patient_id"].count()
